In [1]:
import json 
import networkx as nx
  

from conjecture.consolidated_functions import *
from conjecture.all_valid_orderings import *
from conjecture.optimization_functions import *
from iterative_heuristic.modified_etf import Mod_ETF
from iterative_heuristic.approximate_speeds import *
from iterative_heuristic.consolidated import *
from makespan_energy.construct_graph_util import *
from makespan_energy.visualization_util import *
import networkx as nx
import numpy as np
import math
import plotly.express as px
import pandas as pd 

In [2]:
# Construct DAGs

file_list = ['seismology-chameleon-100p-001.json',
'seismology-chameleon-200p-001.json',
'seismology-chameleon-300p-001.json',
'seismology-chameleon-400p-001.json',
'seismology-chameleon-500p-001.json',
'seismology-chameleon-600p-001.json',
'seismology-chameleon-700p-001.json',
'seismology-chameleon-800p-001.json',
'seismology-chameleon-900p-001.json',
'seismology-chameleon-1000p-001.json',
'seismology-chameleon-1100p-001.json']
    
G_list = []

for file in file_list:
    
    # Opening JSON file 
    f = open('pegasus-traces-master/seismology/chameleon-cloud/' + file) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 

    taskname_to_id = {}
    id_count = 0

    G = nx.DiGraph()

    for task in data['workflow']['jobs']:

        child_id = id_count
        id_count += 1
        taskname_to_id[task['name']] = child_id

        if not G.has_node(child_id):
                G.add_node(child_id)

        for parent in task['parents']:
            if parent in taskname_to_id:

                parent_id = taskname_to_id[parent]

            else:

                parent_id = id_count
                id_count += 1
                taskname_to_id[parent] = parent_id


            if not G.has_edge(parent_id, child_id):
                    G.add_edge(parent_id, child_id)
    print(len(G.nodes))

    
   
    G_list.append(G)
    
    # Closing file 
    f.close() 

101
201
301
401
501
601
701
801
901
1001
1101


In [3]:
mrt = True
tie_breaking_rule = 2
plot = False

x = []
y = []
lbs= []
ubs = []

y_optimal = []
y_lb = []
bad_one = 0
box_whiskers_data = []
num_machines = 3
naives1_ratio = []
naives2_ratio = []
heur_ratio = []
for G in G_list:
    
    
   
    num_tasks = len(G.nodes)
    
    w = [1 for _ in range(num_tasks)]
    s = [1 for _ in range(num_tasks)]

    naive1, heuristic, _ = iterative_and_naive_heuristic_no_ratio_reversed(num_machines, w, G, naive_version=1, iterations=1)
    naive2, _, _ = iterative_and_naive_heuristic_no_ratio_reversed(num_machines, w, G, naive_version=2, iterations=1)
    _, _, relaxed_opt, _ = get_optimal_schedule(mrt, G, num_machines, w, plot, verbose=False)



    naives1_ratio.append(naive1/relaxed_opt)
    naives2_ratio.append(naive2/relaxed_opt)
    heur_ratio.append(heuristic/relaxed_opt)
   
    # y.append(obj_opt)
    # lbs.append(d["objective_task_scaling"])
#     ub = obj_ub_precise = 2 * sum(psize_to_speed(ub_lst_graph(G, order)))
#     ubs.append(ub)


apm 104.5.118.159_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          106
   Constants    :            0
   Variables    :         1323
   Intermediates:            0
   Connections  :          914
   Equations    :          813
   Residuals    :          813
 
 Number of state variables:           1323
 Number of total equations: -          918
 Number of slack variables: -          305
 ---------------------------------------
 Degrees of freedom       :            100
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con

apm 104.5.118.159_gk_model1 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          206
   Constants    :            0
   Variables    :         2623
   Intermediates:            0
   Connections  :         1814
   Equations    :         1613
   Residuals    :         1613
 
 Number of state variables:           2623
 Number of total equations: -         1818
 Number of slack variables: -          605
 ---------------------------------------
 Degrees of freedom       :            200
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con

apm 104.5.118.159_gk_model2 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          306
   Constants    :            0
   Variables    :         3923
   Intermediates:            0
   Connections  :         2714
   Equations    :         2413
   Residuals    :         2413
 
 Number of state variables:           3923
 Number of total equations: -         2718
 Number of slack variables: -          905
 ---------------------------------------
 Degrees of freedom       :            300
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con

apm 104.5.118.159_gk_model3 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          406
   Constants    :            0
   Variables    :         5223
   Intermediates:            0
   Connections  :         3614
   Equations    :         3213
   Residuals    :         3213
 
 Number of state variables:           5223
 Number of total equations: -         3618
 Number of slack variables: -         1205
 ---------------------------------------
 Degrees of freedom       :            400
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con

apm 104.5.118.159_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          506
   Constants    :            0
   Variables    :         6523
   Intermediates:            0
   Connections  :         4514
   Equations    :         4013
   Residuals    :         4013
 
 Number of state variables:           6523
 Number of total equations: -         4518
 Number of slack variables: -         1505
 ---------------------------------------
 Degrees of freedom       :            500
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con

...

apm 104.5.118.159_gk_model7 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          806
   Constants    :            0
   Variables    :        10423
   Intermediates:            0
   Connections  :         7214
   Equations    :         6413
   Residuals    :         6413
 
 Number of state variables:          10423
 Number of total equations: -         7218
 Number of slack variables: -         2405
 ---------------------------------------
 Degrees of freedom       :            800
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con

apm 104.5.118.159_gk_model8 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          906
   Constants    :            0
   Variables    :        11723
   Intermediates:            0
   Connections  :         8114
   Equations    :         7213
   Residuals    :         7213
 
 Number of state variables:          11723
 Number of total equations: -         8118
 Number of slack variables: -         2705
 ---------------------------------------
 Degrees of freedom       :            900
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con

apm 104.5.118.159_gk_model9 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :         1006
   Constants    :            0
   Variables    :        13023
   Intermediates:            0
   Connections  :         9014
   Equations    :         8013
   Residuals    :         8013
 
 Number of state variables:          13023
 Number of total equations: -         9018
 Number of slack variables: -         3005
 ---------------------------------------
 Degrees of freedom       :           1000
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con

Results
0 Speed: [1.0004998377] Ending Time: [0.9995006009] Interval: [0.0, 0.9995] Task process time: 0.9995
1 Speed: [1.0004998377] Ending Time: [0.99950060091] Interval: [0.0, 0.9995] Task process time: 0.9995
2 Speed: [1.0004998377] Ending Time: [0.9995006009] Interval: [0.0, 0.9995] Task process time: 0.9995
3 Speed: [1.0004998377] Ending Time: [0.99950060091] Interval: [0.0, 0.9995] Task process time: 0.9995
4 Speed: [1.0004998377] Ending Time: [0.99950060091] Interval: [0.0, 0.9995] Task process time: 0.9995
5 Speed: [1.0004998377] Ending Time: [0.99950060091] Interval: [0.0, 0.9995] Task process time: 0.9995
6 Speed: [1.0004998377] Ending Time: [0.99950060091] Interval: [0.0, 0.9995] Task process time: 0.9995
7 Speed: [1.0004998377] Ending Time: [0.99950060091] Interval: [0.0, 0.9995] Task process time: 0.9995
8 Speed: [1.0004998378] Ending Time: [0.99950060088] Interval: [0.0, 0.9995] Task process time: 0.9995
9 Speed: [1.0004998377] Ending Time: [0.99950060091] Interval: [0.0

apm 104.5.118.159_gk_model10 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :         1106
   Constants    :            0
   Variables    :        14323
   Intermediates:            0
   Connections  :         9914
   Equations    :         8813
   Residuals    :         8813
 
 Number of state variables:          14323
 Number of total equations: -         9918
 Number of slack variables: -         3305
 ---------------------------------------
 Degrees of freedom       :           1100
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co

In [4]:
# print("x is \n", x)
# print("ubs are \n", ubs)
# print("naives1 are \n",naives1)
# print("naives2 are \n", naives2)
# print("heurs are \m",heur)

In [5]:
ratios1 = [n/h for n,h in zip(naives1, heur)]
ratios2 = [n/h for n,h in zip(naives2, heur)]


NameError: name 'naives1' is not defined

In [ ]:
#x =[101, 201, 301, 401, 501, 601, 701, 801, 901, 1001, 1101]
# ratios1 = [0.999788350235788, 1.0000743145065232, 1.0001471358437553, 0.9999861540674297, 1.000020231865563, 1.0000540328317853, 0.9999954488038859, 1.000010249842991, 1.000029883645439, 0.9999977639726605, 1.0000064437046141]
# ratios2 = [1.7299089173533906, 2.3072848516706763, 2.768371998515514, 3.1628507489592366, 3.5137347400404892, 3.832758177255638, 4.126960426714786, 4.401744554980148, 4.660415907068095, 4.905282530007631, 5.138634468027188]


In [ ]:
import matplotlib.pyplot as plt
import math

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])


#print(ub_ratio)
#plt.scatter(obj_lb_lst[:,0], obj_lb_lst[:,1], label='lower bound', color="g", linewidths=0.1)


# plt.scatter(x, y, s=5, label='opt/lb', color='b')
plt.scatter(x, naives1_ratio, label='Naive1 [Number of tasks remaining on same machine] / relaxed OPT*', color='k')
plt.scatter(x, naives2_ratio, label='Naive2 [number of descendants] / relaxed OPT*', color='r')
#plt.scatter(x, naives, label='Naives', color='g')
plt.xlabel('Number of tasks')
plt.ylabel('Ratios')
plt.title("Seismology: Ratio of objectives of Naives to relaxed OPT* (Naive/Heur)")
plt.legend()
plt.show()

In [ ]:
import csv
with open("seismology_both_naives_heur_3_machines.csv", 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(["task_size", "naive1", "heuristic", "upperbound"])
    writer.writerows(zip(x,  naives1, naives2, heur,  ubs))